In [1]:
!pip install yfinance
!pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29106 sha256=d4a69140d2e9860bea9094d63485fa8577b69a9321aa921f1105569e866c4a38
  Stored in directory: /root/.cache/pip/wheels/1b/81/50/db21d9643326c61dd6fee1dc2562d8351ee7753164c24ead38
Successfully built ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Raw Package
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import datetime
from ta.trend import MACD 

# Market Data 
import yfinance as yf

#Graphing/Visualization
import datetime as dt 
import plotly.graph_objs as go 

# Override Yahoo Finance 
yf.pdr_override()

# Create input field for our desired stock 
stock=input("Enter a stock ticker symbol: ")

# Retrieve stock data frame (df) from yfinance API at an interval of 1m 
df = yf.download(tickers=stock,period='4y',interval='1d')

print(df)

# Declare plotly figure (go)
fig = go.Figure()
fig1 = go.Figure()
fig2 = go.Figure()

df['MA50'] = df['Close'].ewm(span=50, adjust=False).mean()
df['MA100'] = df['Close'].ewm(span=100, adjust=False).mean()
df['MA200'] = df['Close'].ewm(span=200, adjust=False).mean()

macd = MACD(close=df['Close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)

fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                name='market data'))

fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA50'], 
                         opacity=0.7, 
                         line=dict(color='blue', width=2), 
                         name='MA50'))

fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA100'], 
                         opacity=0.7, 
                         line=dict(color='orange', width=2), 
                         name='MA100'))

fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA200'], 
                         opacity=0.7, 
                         line=dict(color='yellow', width=2), 
                         name='MA200'))

fig.update_layout(
    title= str(stock)+' Live Share Price:',
    yaxis_title='Stock Price (USD per Shares)')               

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15d", step="day", stepmode="backward"),
            dict(count=30, label="30d", step="day", stepmode="backward"),
            dict(count=1, label="HTD", step="month", stepmode="todate"),
            dict(count=3, label="3mo", step="month", stepmode="backward"),
            dict(count=6, label="6mo", step="month", stepmode="backward"),
            dict(step="all")
        ])
    )
)

# Plot volume trace on 1st row
colors = ['green' if row['Open'] - row['Close'] >= 0 
          else 'red' for index, row in df.iterrows()]
fig1.add_trace(go.Bar(x=df.index, 
                     y=df['Volume'],
                     marker_color=colors))

# Plot MACD trace on 2nd row
fig2.add_trace(go.Bar(x=df.index, 
                     y=macd.macd_diff(),
                     marker_color='orange',
                     name='MACD Histogram'))

fig2.add_trace(go.Scatter(x=df.index,
                    y=macd.macd(),
                    line=dict(color='black', width=2),
                    name='MACD line (EMA-26/SLOW)'))

fig2.add_trace(go.Scatter(x=df.index,
                    y=macd.macd_signal(),
                    line=dict(color='blue', width=1),
                    name='Signal line (EMA-12/FAST)'))

fig1.update_yaxes(title_text="Volume")

fig2.update_yaxes(title_text="MACD", showgrid=False)

fig1.update_layout(title='Volume') 

fig2.update_layout(title='MACD(12,26,9)') 

fig1.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15d", step="day", stepmode="backward"),
            dict(count=30, label="30d", step="day", stepmode="backward"),
            dict(count=1, label="HTD", step="month", stepmode="todate"),
            dict(count=3, label="3mo", step="month", stepmode="backward"),
            dict(count=6, label="6mo", step="month", stepmode="backward"),
            dict(count=12, label="12mo", step="month", stepmode="backward"),
            dict(count=24, label="24mo", step="month", stepmode="backward")
        ])
    )
)

fig2.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15d", step="day", stepmode="backward"),
            dict(count=30, label="30d", step="day", stepmode="backward"),
            dict(count=1, label="HTD", step="month", stepmode="todate"),
            dict(count=3, label="3mo", step="month", stepmode="backward"),
            dict(count=6, label="6mo", step="month", stepmode="backward"),
            dict(count=12, label="12mo", step="month", stepmode="backward"),
            dict(count=24, label="24mo", step="month", stepmode="backward")
        ])
    )
)

fig.show()
fig1.show()
fig2.show()

Enter a stock ticker symbol: BTC-USD
[*********************100%***********************]  1 of 1 completed
                    Open          High           Low         Close  \
Date                                                                 
2018-12-18   3544.761475   3701.349365   3487.169189   3696.059082   
2018-12-19   3706.824951   3949.322998   3687.229980   3745.950684   
2018-12-20   3742.195068   4191.228516   3728.974609   4134.441406   
2018-12-21   4133.703613   4198.429688   3850.946289   3896.543701   
2018-12-22   3898.083740   4014.182617   3855.739014   4014.182617   
...                  ...           ...           ...           ...   
2022-12-14  17782.066406  18318.531250  17739.513672  17815.650391   
2022-12-15  17813.644531  17846.744141  17322.589844  17364.865234   
2022-12-16  17364.546875  17505.525391  16584.701172  16647.484375   
2022-12-17  16646.982422  16800.589844  16614.029297  16795.091797   
2022-12-18  16793.917969  16793.917969  16729.806641  

In [3]:
# import modules
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense,Activation
import matplotlib.pyplot as plt
import math

In [4]:
df = df.reset_index()
df

,Date,Open,High,Low,Close,Adj Close,Volume,MA50,MA100,MA200
0,2018-12-18,3544.761475,3701.349365,3487.169189,3696.059082,3696.059082,5911325473,3696.059082,3696.059082,3696.059082
1,2018-12-19,3706.824951,3949.322998,3687.229980,3745.950684,3745.950684,6810689119,3698.015615,3697.047035,3696.555516
2,2018-12-20,3742.195068,4191.228516,3728.974609,4134.441406,4134.441406,8927129279,3715.130352,3705.708309,3700.912589
3,2018-12-21,4133.703613,4198.429688,3850.946289,3896.543701,3896.543701,7206015706,3722.244601,3709.487228,3702.859168
4,2018-12-22,3898.083740,4014.182617,3855.739014,4014.182617,4014.182617,5605823233,3733.693151,3715.520800,3705.956913
...,...,...,...,...,...,...,...,...,...,...
1457,2022-12-14,17782.066406,18318.531250,17739.513672,17815.650391,17815.650391,25534481470,17671.127748,18904.834310,22369.965337
1458,2022-12-15,17813.644531,17846.744141,17322.589844,17364.865234,17364.865234,20964448341,17659.117453,18874.339873,22320.163346
1459,2022-12-16,17364.546875,17505.525391,16584.701172,16647.484375,16647.484375,24031608960,17619.445568,18830.243725,22263.718780
1460,2022-12-17,16646.982422,16800.589844,16614.029297,16795.091797,16795.091797,14463581825,17587.117969,18789.943687,22209.304581


In [5]:
train_dates = pd.to_datetime(df['Date']).dt.date
train_dates

0       2018-12-18
1       2018-12-19
2       2018-12-20
3       2018-12-21
4       2018-12-22
           ...    
1457    2022-12-14
1458    2022-12-15
1459    2022-12-16
1460    2022-12-17
1461    2022-12-18
Name: Date, Length: 1462, dtype: object

In [6]:
df1 = df[['Close', 'Open', 'High', 'Low']]
df1

,Close,Open,High,Low
0,3696.059082,3544.761475,3701.349365,3487.169189
1,3745.950684,3706.824951,3949.322998,3687.229980
2,4134.441406,3742.195068,4191.228516,3728.974609
3,3896.543701,4133.703613,4198.429688,3850.946289
4,4014.182617,3898.083740,4014.182617,3855.739014
...,...,...,...,...
1457,17815.650391,17782.066406,18318.531250,17739.513672
1458,17364.865234,17813.644531,17846.744141,17322.589844
1459,16647.484375,17364.546875,17505.525391,16584.701172
1460,16795.091797,16646.982422,16800.589844,16614.029297


In [7]:
df2 = df1.mean(axis=1)
df2

0        3607.334778
1        3772.332153
2        3949.209900
3        4019.905823
4        3945.546997
            ...     
1457    17913.940430
1458    17586.960938
1459    17025.564453
1460    16714.173340
1461    16769.816895
Length: 1462, dtype: float64

In [8]:
df3 = np.reshape(df2.values, (len(df2),1))
scaler = MinMaxScaler((0, 1))
df4 = scaler.fit_transform(df3)

In [9]:
train_d = int(len(df4) * 0.8)
test_d = len(df4) - train_d
train_d, test_d = df4[0:train_d,:], df4[train_d:len(df4),:]

In [10]:
def new_dataset(dataset, step_size):
    data_X, data_Y = [], []
    for i in range(len(dataset)-step_size-1):
        a = dataset[i:(i+step_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + step_size, 0])
    return np.array(data_X), np.array(data_Y)
step_size = 15
X_train, Y_train = new_dataset(train_d, step_size)
X_test, Y_test = new_dataset(test_d, step_size)

In [11]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(step_size,1)))
model.add(LSTM(units=50))
model.add(Dense(1))

In [12]:
model.compile(loss='mean_squared_error', optimizer='adam')
history=model.fit(X_train,Y_train,epochs=10,batch_size=32)

Epoch 1/10
37/37 [==============================] - 5s 16ms/step - loss: 0.0179
Epoch 2/10
37/37 [==============================] - 1s 16ms/step - loss: 0.0020
Epoch 3/10
37/37 [==============================] - 1s 16ms/step - loss: 0.0017
Epoch 4/10
37/37 [==============================] - 1s 16ms/step - loss: 0.0016
Epoch 5/10
37/37 [==============================] - 1s 17ms/step - loss: 0.0016
Epoch 6/10
37/37 [==============================] - 1s 17ms/step - loss: 0.0016
Epoch 7/10
37/37 [==============================] - 1s 15ms/step - loss: 0.0014
Epoch 8/10
37/37 [==============================] - 1s 16ms/step - loss: 0.0014
Epoch 9/10
37/37 [==============================] - 1s 38ms/step - loss: 0.0014
Epoch 10/10
37/37 [==============================] - 1s 35ms/step - loss: 0.0012


In [13]:
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)

9/9 [==============================] - 0s 9ms/step


In [14]:
trainScore = math.sqrt(mean_squared_error(Y_train, trainPredict))
print('Train RMSE: %.2f' % (trainScore))

testScore = math.sqrt(mean_squared_error(Y_test, testPredict))
print('Test RMSE: %.2f' % (testScore))

Train RMSE: 0.04
Test RMSE: 0.03


In [15]:
trainPredictY = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([Y_train])
testPredictY = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([Y_test])

In [16]:
n_future = 10
y_future = []

x_pred = X_test[-1:, :]       # last observed input sequence
y_pred = testPredict[-1]         # last observed target value

for i in range(n_future):

    # feed the last forecast back to the model as an input
    x_pred = np.append(x_pred[:, 1:], y_pred.reshape(1, 1), axis=1)

    # generate the next forecast
    y_pred = model.predict(x_pred)

    # save the forecast
    y_future.append(y_pred.flatten()[0])

1/1 [==============================] - 0s 18ms/step


In [17]:
y_future = np.array(y_future).reshape(-1, 1)
y_future_S = scaler.inverse_transform(y_future)
y_future_S

array([[16997.816],
       [16983.945],
       [16956.576],
       [16920.082],
       [16878.48 ],
       [16833.67 ],
       [16786.2  ],
       [16736.943],
       [16687.645],
       [16638.281]], dtype=float32)

In [18]:
from datetime import date
end = date.today()

start = date(end.year, end.month-1, end.day)

new = date(end.year, end.month, end.day+1)

last = date(end.year, end.month, end.day+10)


In [19]:
dff  = pd.DataFrame(columns=['Date', 'Average price predicted for the next 10 days'])
dff['Date'] = pd.date_range(end+ pd.Timedelta(days=1), periods=n_future)
dff['Average price predicted for the next 10 days'] = y_future_S.flatten()
dff.reset_index()
dff

,Date,Average price predicted for the next 10 days
0,2022-12-19,16997.816406
1,2022-12-20,16983.945312
2,2022-12-21,16956.576172
3,2022-12-22,16920.082031
4,2022-12-23,16878.480469
5,2022-12-24,16833.669922
6,2022-12-25,16786.199219
7,2022-12-26,16736.943359
8,2022-12-27,16687.644531
9,2022-12-28,16638.281250


In [20]:
data = df1.copy()
data['Average price (actual)'] = data.mean(numeric_only=True, axis=1)
ext_col = df["Date"]
data.insert(0, "Date", ext_col)
data.drop(['Close', 'Open', 'High', 'Low'], axis = 1, inplace = True) 
data

,Date,Average price (actual)
0,2018-12-18,3607.334778
1,2018-12-19,3772.332153
2,2018-12-20,3949.209900
3,2018-12-21,4019.905823
4,2018-12-22,3945.546997
...,...,...
1457,2022-12-14,17913.940430
1458,2022-12-15,17586.960938
1459,2022-12-16,17025.564453
1460,2022-12-17,16714.173340


In [21]:
mask = (train_dates > start)
data1 = data.loc[mask]
data1.reset_index()
data1

,Date,Average price (actual)
1432,2022-11-19,16694.013672
1433,2022-11-20,16500.055908
1434,2022-11-21,15992.194580
1435,2022-11-22,15970.431152
1436,2022-11-23,16403.748047
1437,2022-11-24,16622.335938
1438,2022-11-25,16528.958008
1439,2022-11-26,16517.237305
1440,2022-11-27,16484.985352
1441,2022-11-28,16298.752197


In [22]:
frames = [data1, dff]
result = pd.concat(frames)
display(result)

,Date,Average price (actual),Average price predicted for the next 10 days
1432,2022-11-19,16694.013672,NaN
1433,2022-11-20,16500.055908,NaN
1434,2022-11-21,15992.194580,NaN
1435,2022-11-22,15970.431152,NaN
1436,2022-11-23,16403.748047,NaN
1437,2022-11-24,16622.335938,NaN
1438,2022-11-25,16528.958008,NaN
1439,2022-11-26,16517.237305,NaN
1440,2022-11-27,16484.985352,NaN
1441,2022-11-28,16298.752197,NaN


In [23]:
import plotly.express as px 
# plotting the line chart
figx = px.scatter(result, x="Date", y=["Average price (actual)", "Average price predicted for the next 10 days"])
figx.update_yaxes(title_text="Price") 
# showing the plot
figx.show()